In [37]:
import pandas as pd
import nltk
import re
import string
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize

In [32]:
file_path = '443.csv'
df = pd.read_csv(file_path, usecols = ['id','title','description','likecount','dislikecount','tags'])
df = df.dropna()

In [33]:
regex = re.compile('[%s]' % re.escape(string.punctuation))
words = set(nltk.corpus.words.words())
words = {x.lower() for x in words}
stop_words = set(stopwords.words('english'))
ps = PorterStemmer()
def rm_punctuation(sent):
    return regex.sub('',sent)
def rm_foreign(sent, min_words = 20):
    original = sent
    sent = sent.replace('\r\n', '')
    sent = sent.replace('\n', '')
    sent = re.sub(r'href\S+', '', sent)
    check = " ".join(w for w in nltk.wordpunct_tokenize(sent) \
         if (w.lower() in words) and (w.lower() not in stop_words))
    # if new title or description contains too little infromation afte cleaning, drop it.
    if len(check) / len(original) > 0.3:
        if len(check) > min_words:
            return check.lower()
        else:
            return None
        return check.lower()
    else:
        return None
    
def tags_cleaning(tags):
    tags_lst = tags.split(',')
    new_set = set()
    for tag in tags_lst:
        new_tag = ''
        for w in nltk.wordpunct_tokenize(tag):
            if (w.lower() in words) and (w.lower() not in stop_words):
                new_tag += w 
                new_tag += " "
        if len(new_tag.strip()) / len(tag) < 0.5:
            new_tag = ''
        new_tag = stem_condense(new_tag)
        if new_tag !='' and (not new_tag.isspace()): # drop all tags that is none or only contain spaces .
            new_set.update({new_tag.strip().lower()})
        
    if len(new_set) == 0:
        return None
    return list(new_set)[0:5]

# generate synonyms dictionary "dic"
syn = pd.read_csv('synonyms.csv') # this file is from github project
dic = dict()
for ind in syn.index:
    row = syn.iloc[ind,:]['words'].split(',')
    for i in range(1,len(row)):      
        dic[row[i].strip()] = row[0]

def stem_condense(sent):
    new_sent = []
    for w in word_tokenize(sent):
        #condense
        condensed_w = dic.get(w,w)
        #stem
        stem_w = ps.stem(condensed_w)
        new_sent.append(stem_w)
    new_sent = ' '.join(new_sent)
    return new_sent

In [34]:
# This cell should only be run ONCE!!
df['description'] = df['description'].apply(rm_foreign)
df = df.dropna()
df['description'] = df['description'].apply(rm_punctuation)
df['tags'] = df['tags'].apply(tags_cleaning)
df = df.dropna()
df = df.reset_index(drop=True)

In [35]:
# write in a csv file.
# df.to_csv('cleaned_condensed.csv')

In [ ]:
# write in a csv file.
# df.to_csv('cleaned_condensed.csv')
import csv
df.head()
header=df['tags']
result=[]
total_rows = df['id'].count()

result.append(['Id','Title','Description','LikeCount','DislikeCount','Condensed Tags (comma delimited string)'])
for i in range(total_rows):
    result.append([df['id'][i],df['title'][i],df['description'][i],df['likecount'][i],df['dislikecount'][i]])
    for j in range(len(df['tags'][i])):
        result[i+1].append(df['tags'][i][j])
with open('datafile.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(result)